                                  PROJECT_C4

                               
                                   TEAM-76
 

                            KRISHNA KOTARU(bkotaru)  
                            GANESH THANU(gthanu)
                            SHAUNAK PATEL(shpate25)

In [8]:
# This cell handles the merge function
import pandas as pd
import os
def merge_files(filex,filey):
    #Read files
    x_df = pd.read_csv(filex)
    y_df = pd.read_csv(filey)

    #Extract column integer values
    x_first = pd.DataFrame(x_df.columns).T
    y_first = pd.DataFrame(y_df.columns).T

    #Define column names
    x_columns = ["a1", "a2", "a3", "g1", "g2", "g3"]
    y_columns = ["y"]

    #Assign column names
    x_df.columns = x_columns
    y_df.columns= y_columns
    x_first.columns = x_columns
    y_first.columns = y_columns

    #Concatenate the dataframes with new column names
    x_df = pd.concat([x_first, x_df],axis = 0).reset_index(drop=True)
    y_df = pd.concat([y_first, y_df],axis = 0).reset_index(drop=True)

    #Define the output dataframe structure
    combined_df = pd.DataFrame(columns=["a1", "a2", "a3", "g1", "g2", "g3", "y"])
    y_ind = 0

    #Loop through and merge the dataframes
    for i in range(0,len(x_df)):
        if i%4 == 0: 
            y = y_df.iloc[y_ind:y_ind+1].reset_index(drop=True)
            y_ind += 1
        x = x_df.iloc[i:i+1].reset_index(drop=True)
        z = pd.concat([x,y],axis=1).reset_index(drop=True)
        combined_df = combined_df.append(z, ignore_index = True)
    combined_df = combined_df.astype(float)
    return combined_df



In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
# This cell contains the code for the implementation of sliding window

from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from statistics import mode
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
def split_sequence_slide(Xval,yval,n_steps):
  X , y = list(), list()
  one_hot_dict = {3 : [0,0,0,1], 2 : [0,0,1,0], 1 : [0,1,0,0], 0 : [1,0,0,0]}
  count = 0
  while(count <= len(Xval)-n_steps):
    arr_of_X_60 = np.array(Xval[count: count + n_steps,:], dtype=np.float32)
    arr_of_y_60 = np.array(yval.iloc[count: count + n_steps, :])
    count = count + 1

    class_labels = {3:0,2:0,1:0,0:0}
    for arr_y in arr_of_y_60:
      if len(arr_y) > 0:
        val = list(arr_y).index(1)
        class_labels[val]+=1
        
    mode = max(class_labels, key=class_labels.get)
    y.append(np.array(one_hot_dict[mode]))
    X.append(arr_of_X_60)
    
    
  return X,y

In [ ]:
# This cell involves extracting values from the files, normalizing/encoding them  

from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

  
dict= {1:8, 2:5, 3:3, 4:2, 5:3 ,6:3, 7:4, 8:1 }
one_hot_dict = {3 : [0,0,0,1], 2 : [0,0,1,0], 1 : [0,1,0,0], 0 : [1,0,0,0]}


model_input_x,model_input_y = [],[]
for i in dict.keys():
    for j in range(1,dict[i]+1):
        XInput = "TrainingData/subject_00"+str(i)+"_0"+str(j)+"__x.csv"
        YInput = "TrainingData/subject_00"+str(i)+"_0"+str(j)+"__y.csv"

        output_df = merge_files(XInput,YInput)

        X,y = output_df[['a1','a2','a3','g1','g2','g3']],output_df['y']
        y = y.fillna(0)
        y = y.astype(int)
        y_new = []
        # Onehot encoding the output values
        for class_label in y:
            y_new.append(np.array(one_hot_dict[class_label]))
        y_new = np.array(y_new)
        y_new = pd.DataFrame(y_new)
       
        # Normalization
        scaler = StandardScaler()
        scaler.fit(X)
        X_normal = scaler.transform(X)
  
        # Windowing
        X_windowed,y_windowed = split_sequence_slide(X_normal,y_new,60)
  
        X_windowed = np.array(X_windowed)
        y_windowed = np.array(y_windowed)
        
        model_input_x.append(X_windowed)
        model_input_y.append(y_windowed)
        
        print(f"sub{i} session{j}")
      
        print("\n")
       

In [20]:
X=[]
for arr in model_input_x:
    for i in arr:
        X.append(i)
Y=[]
for arr in model_input_y:
    for i in arr:
        Y.append(i)

In [16]:
# Computing sample weights
from sklearn.utils import compute_sample_weight   
weights = compute_sample_weight(class_weight = 'balanced', y = Y)    

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Dropout
import numpy

model_5 = Sequential()
model_5.add(Bidirectional(LSTM(6, activation='relu', input_shape= (60,6),return_sequences=True)))
model_5.add(Bidirectional(LSTM(12,dropout=0.5,recurrent_dropout=0.5,activation='relu')))
model_5.add(Dense(24, activation='relu'))
model_5.add(Dropout(.5))
model_5.add(Dense(4, activation='softmax'))
model_5.compile(optimizer='adam', loss='categorical_crossentropy')

x_train = tf.cast(numpy.array(X) , dtype=tf.float32)
y_train = tf.cast(numpy.array(Y) , dtype=tf.int32)
model_5.fit(x_train,y_train,batch_size=64, epochs = 10, sample_weight=weights )

In [ ]:
# Run only if needed
model_5 = tf.keras.models.load_model('model_4.keras')
print(model_5.summary)

In [ ]:
#This cell handles output generation and storage

dict_prediction ={9:1,10:1,11:1,12:1}

for l in dict_prediction.keys():
    for j in range(1,dict_prediction[l]+1):
        
        x_df = pd.read_csv("TestData/subject_0"+str(l)+"_0"+str(j)+"__x.csv")
        timestamp_length = len(pd.read_csv("TestData/subject_0"+str(l)+"_0"+str(j)+"__y_time.csv"))
        x_first = pd.DataFrame(x_df.columns).T
        
        x_columns = ["a1", "a2", "a3", "g1", "g2", "g3"]
        x_df.columns = x_columns
        x_first.columns = x_columns
        x_df = pd.concat([x_first, x_df],axis = 0).reset_index(drop=True)

        X = []
        count = 0
        #Normalization of input values
        scaler = StandardScaler()
        scaler.fit(x_df)
        X_normal = scaler.transform(x_df)

        # Windowing
        while(count <= len(x_df)-60):
            arr_of_X_60 = np.array(X_normal[count: count + 60,:], dtype=np.float32)
            count = count + 1
            X.append(arr_of_X_60)

        X = numpy.array(X)
        # Prediction 
        model_output_y = model_5.predict(X)
        # Assingning 1 to the class with the highest probabaility
        model_output_y_new = tf.one_hot(tf.argmax(model_output_y,axis =1), depth = 4)
        model_output_y_new = model_output_y_new.numpy()
        model_output_y_new = model_output_y_new.astype(np.int64)

        #Choosing one class for every 4 input sets because of the frequency of timestamps 
        pred_output =[]
        for i in range(len(model_output_y_new)):
            if i%4==0:
                pred_output.append(list(model_output_y_new[i]).index(1))
        #Assigning the last predicted value to the remaining classes if any
        diff = timestamp_length - len(pred_output)
        val = pred_output[-1]
        for i in range(diff):
            pred_output.append(val)


        y_arr = numpy.array(pred_output)
        y_arr = y_arr.transpose()
        # Creating a csv file with the predicted values
        import csv
        with open("subject_0"+str(l)+"_0"+str(j)+"__y.csv", mode = 'w',newline='') as f:
            w = csv.writer(f)
            w.writerows(map(lambda x:[x],y_arr))
                
